In [1]:
from configparser import ConfigParser
import spotipy
import sys
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
import numpy as np

parser = ConfigParser()
parser.read('./spotify_credentials.cfg')

SPOTIPY_CLIENT_ID = parser.get('spotify', 'SPOTIPY_CLIENT_ID')
SPOTIPY_CLIENT_SECRET = parser.get('spotify', 'SPOTIPY_CLIENT_SECRET')

sp = spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials(client_id=SPOTIPY_CLIENT_ID, client_secret=SPOTIPY_CLIENT_SECRET))

In [15]:
input = pd.read_csv('artists.csv', header = None)
artist_ids = []
for link in input[0]:
    
    artist_ids.append(link[32:54])

artist_ids

['4tZwfgrHOc3mvqYlEYSvVi',
 '2CIMQHirSU0MQqyYHq0eOx',
 '6nxWCVXbOlEVRexSbLsTer',
 '5K4W6rqBFWDnAN6FQUkS6x',
 '0QWrMNukfcVOmgEU0FEDyD',
 '4LEiUm1SRbFMgfqnQTwUbQ',
 '06HL4z0CvFAxyc27GXpf02',
 '0oeUpvxWsC8bWS6SnpU8b9',
 '3Qy1IxDSU8SLpUUOfbOpxM',
 '5cIc3SBFuBLVxJz58W2tU9',
 '4gzpq5DPGxSnKTe4SA8HAU',
 '1dWEYMPtNmvSVaDNLgB6NV',
 '3iOvXCl6edW5Um0fXEBRXy',
 '26T3LtbuGT1Fu9m0eRq5X3',
 '0pCNk4D3E2xtszsm6hMsWr',
 '3YQKmKGau1PzlVlkL1iodx',
 '3AA28KZvwAUcZuOKwyblJQ',
 '2cFrymmkijnjDg9SS92EPM',
 '7pXu47GoqSYRajmBCjxdD6',
 '4yvcSjfu4PC0CYQyLy4wSq']

In [16]:
albums = []
album_track_ids = []
album_ids = []
artist_uri = 'spotify:artist:' + artist_ids[0]

# Add album ids to album_ids list
results = sp.artist_albums(artist_uri, album_type='album')
albums.extend(results['items'])
while results['next']:
    results = sp.next(results)
    albums.extend(results['items'])
for album in albums:
    album_ids.append(album['id'])

all_tracks = []
for id in album_ids:

    track_ids = []

    results = sp.album_tracks(id)
    tracks = results['items']
    all_tracks.extend(tracks)
    for track in tracks:
        track_ids.append(track['id'])

    album_track_ids.append(track_ids)


assert len(album_track_ids) == len(albums)

len(albums)

21

In [26]:
# sp.audio_features('2RpXUwCytDFqZSHl6kIquB')
sp.audio_analysis('2RpXUwCytDFqZSHl6kIquB')['sections']

[{'start': 0.0,
  'duration': 4.51881,
  'confidence': 1.0,
  'loudness': -18.241,
  'tempo': 0.0,
  'tempo_confidence': 0.0,
  'key': 1,
  'key_confidence': 0.0,
  'mode': 1,
  'mode_confidence': 0.0,
  'time_signature': 3,
  'time_signature_confidence': 1.0},
 {'start': 4.51881,
  'duration': 13.66397,
  'confidence': 1.0,
  'loudness': -7.513,
  'tempo': 122.011,
  'tempo_confidence': 0.095,
  'key': 2,
  'key_confidence': 0.406,
  'mode': 1,
  'mode_confidence': 0.447,
  'time_signature': 3,
  'time_signature_confidence': 1.0},
 {'start': 18.18277,
  'duration': 22.68308,
  'confidence': 1.0,
  'loudness': -8.188,
  'tempo': 127.008,
  'tempo_confidence': 0.084,
  'key': 2,
  'key_confidence': 0.0,
  'mode': 1,
  'mode_confidence': 0.0,
  'time_signature': 3,
  'time_signature_confidence': 1.0},
 {'start': 40.86585,
  'duration': 8.56668,
  'confidence': 0.168,
  'loudness': -9.684,
  'tempo': 123.305,
  'tempo_confidence': 0.167,
  'key': 3,
  'key_confidence': 0.121,
  'mode': 1,

In [38]:
# columns_to_remove = ['analysis_url', 'type', 'uri', 'track_href']
headings = ['start',
'duration',
'confidence',
'loudness',
'tempo',
'tempo_confidence',
'key',
'key_confidence',
'mode',
'mode_confidence',
'time_signature',
'time_signature_confidence']
data = pd.DataFrame(columns=headings)

# Create DataFrames of features from track ids
i = 0
j = 0
for album in album_track_ids:
    
    length = len(album)
    
    for track in album:
        track_sections = sp.audio_analysis(track)['sections']
        for results in track_sections:
            if results: 
                features = results
            features_matrix = pd.DataFrame.from_records(features, index=[0])
            
            # Remove unneeded columns
            # features_matrix.drop(columns = columns_to_remove, axis = 1, inplace = True)
            # print(features_matrix)

            # Add track title
            track_title = all_tracks[j]['name']
            features_matrix['track_title'] = track_title
            # Add album title
            album_title = albums[i]['name']
            features_matrix['album_title'] = album_title
            # Add artist name
            name = albums[i]['artists'][0]['name']
            features_matrix['album_artist'] = name

            # Add track number and total tracks
            features_matrix['track_number'] = all_tracks[j]['track_number']
            features_matrix['total_tracks'] = length

            # Add album as a row to the data df
            # print(features_matrix,'\n')
            # print(data)
            data = data.append(features_matrix)

        j += 1

    i += 1

# Remove duplicates
data.drop_duplicates(subset=['track_number', 'album_title', 'album_artist'])
data

,start,duration,confidence,loudness,tempo,tempo_confidence,key,key_confidence,mode,mode_confidence,time_signature,time_signature_confidence,track_title,album_title,album_artist,track_number,total_tracks
0,0.00000,18.97740,1.000,-8.062,119.052,0.403,9,0.656,0,0.395,4,1.000,Give Life Back to Music,Random Access Memories,Daft Punk,1.0,13.0
0,18.97740,34.77193,1.000,-11.683,119.029,0.853,9,0.502,0,0.484,4,1.000,Give Life Back to Music,Random Access Memories,Daft Punk,1.0,13.0
0,53.74933,28.22747,0.723,-9.778,119.047,0.754,0,0.303,1,0.383,4,1.000,Give Life Back to Music,Random Access Memories,Daft Punk,1.0,13.0
0,81.97680,32.76324,0.463,-10.560,119.053,0.743,7,0.204,1,0.489,4,1.000,Give Life Back to Music,Random Access Memories,Daft Punk,1.0,13.0
0,114.74005,16.12582,0.252,-7.082,119.135,0.371,9,0.645,0,0.443,4,1.000,Give Life Back to Music,Random Access Memories,Daft Punk,1.0,13.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,213.10489,93.72905,0.119,-5.362,128.711,0.732,8,0.038,1,0.294,4,1.000,Alive,Homework,Daft Punk,15.0,16.0
0,306.83395,9.41939,1.000,-27.237,128.657,0.551,10,0.000,0,0.000,4,1.000,Alive,Homework,Daft Punk,15.0,16.0
0,0.00000,10.74501,1.000,-11.120,111.954,0.700,7,0.093,1,0.249,4,0.944,Funk Ad,Homework,Daft Punk,16.0,16.0
0,10.74501,32.71623,0.355,-7.287,111.832,0.724,10,0.000,0,0.000,4,0.729,Funk Ad,Homework,Daft Punk,16.0,16.0


In [39]:
data['order'] = data['track_number']/data['total_tracks']

data.to_csv('./data/sections.csv')
data

,start,duration,confidence,loudness,tempo,tempo_confidence,key,key_confidence,mode,mode_confidence,time_signature,time_signature_confidence,track_title,album_title,album_artist,track_number,total_tracks,order
0,0.00000,18.97740,1.000,-8.062,119.052,0.403,9,0.656,0,0.395,4,1.000,Give Life Back to Music,Random Access Memories,Daft Punk,1.0,13.0,0.076923
0,18.97740,34.77193,1.000,-11.683,119.029,0.853,9,0.502,0,0.484,4,1.000,Give Life Back to Music,Random Access Memories,Daft Punk,1.0,13.0,0.076923
0,53.74933,28.22747,0.723,-9.778,119.047,0.754,0,0.303,1,0.383,4,1.000,Give Life Back to Music,Random Access Memories,Daft Punk,1.0,13.0,0.076923
0,81.97680,32.76324,0.463,-10.560,119.053,0.743,7,0.204,1,0.489,4,1.000,Give Life Back to Music,Random Access Memories,Daft Punk,1.0,13.0,0.076923
0,114.74005,16.12582,0.252,-7.082,119.135,0.371,9,0.645,0,0.443,4,1.000,Give Life Back to Music,Random Access Memories,Daft Punk,1.0,13.0,0.076923
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,213.10489,93.72905,0.119,-5.362,128.711,0.732,8,0.038,1,0.294,4,1.000,Alive,Homework,Daft Punk,15.0,16.0,0.937500
0,306.83395,9.41939,1.000,-27.237,128.657,0.551,10,0.000,0,0.000,4,1.000,Alive,Homework,Daft Punk,15.0,16.0,0.937500
0,0.00000,10.74501,1.000,-11.120,111.954,0.700,7,0.093,1,0.249,4,0.944,Funk Ad,Homework,Daft Punk,16.0,16.0,1.000000
0,10.74501,32.71623,0.355,-7.287,111.832,0.724,10,0.000,0,0.000,4,0.729,Funk Ad,Homework,Daft Punk,16.0,16.0,1.000000
